In [251]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import akshare as ak
import datetime
from datetime import date, timedelta
import seaborn as sns
import time
import random
import itertools

from fund_tools import *

In [2]:
# Set date variable

today = date.today()
first_day_of_the_week = today - timedelta(days=today.weekday())
# end = start + timedelta(days=6)
print("Today: " + str(today))
print("Start: " + str(first_day_of_the_week))
# print("End: " + str(end))

Today: 2021-09-30
Start: 2021-09-27


In [3]:
start_date = '2015-01-01'
end_date = today

fund_net_value = get_fund_net_worth('001980',start_date=start_date, end_date=end_date, fund_category='open')

In [182]:
def AIP_weekly(code, start_date, end_date, fund_category, fixed_investment, freq='Monday', df=False):
    
    
    fund_net_value = get_fund_net_worth(code, start_date=start_date, end_date=end_date, fund_category=fund_category)
    
    fund_net_value['WeekDay'] = pd.to_datetime(fund_net_value['净值日期']).dt.day_name()
    
    fund_net_value['定投金额(本金)'] = 0
        
    for i in range(len(fund_net_value['WeekDay'])):
         if fund_net_value['WeekDay'].values[i] == freq:
                fund_net_value['定投金额(本金)'][i] = fixed_investment
            
    fund_net_value['累计定投金额(本金)'] = fund_net_value['定投金额(本金)'].cumsum()
    fund_net_value['购买份额'] = fund_net_value['定投金额(本金)']/fund_net_value['单位净值']
    fund_net_value['累计份额'] = fund_net_value['购买份额'].cumsum()
    fund_net_value['平均成本'] = fund_net_value['累计定投金额(本金)']/fund_net_value['累计份额']

    fund_net_value['累计收益'] = (fund_net_value['单位净值'] - fund_net_value['平均成本']) * fund_net_value['累计份额']
    
    start_invest = fund_net_value['定投金额(本金)'].values.nonzero()[0][0]
    fund_net_value['持有天数'] = (fund_net_value['净值日期'] - fund_net_value['净值日期'][start_invest]).dt.days+1
    for i in range(len(fund_net_value['持有天数'])):
        if fund_net_value['持有天数'][i] < 0:
            fund_net_value['持有天数'][i] = 0
    fund_net_value['年化收益率'] = ((fund_net_value['累计收益'] + fund_net_value['累计定投金额(本金)'])/fund_net_value['累计定投金额(本金)'])**(365/fund_net_value['持有天数'])-1

    fund_net_value['累计收益率'] = fund_net_value['累计收益']/fund_net_value['累计定投金额(本金)']
    
    Stat_df = pd.DataFrame({
        '基金代码': code,
        '持有天数': fund_net_value['持有天数'].values[-1],
        '定投时间': freq,
        '定投金额': fixed_investment,
        '分投期数': fund_net_value['累计定投金额(本金)'].values[-1]/fixed_investment,
        '总购买份额' : '%.3f' % fund_net_value['累计份额'].values[-1],
        '平均成本' : '%.3f' % fund_net_value['平均成本'].values[-1],
        '累计收益' : '%.3f' % fund_net_value['累计收益'].values[-1],
        '累计收益率' : '%.3f' % fund_net_value['累计收益率'].values[-1],
        '年化收益率' : '%.3f' % fund_net_value['年化收益率'].values[-1]
    }, index=['Plan'])
    
    if df:
        return fund_net_value
    else:
        return Stat_df

In [195]:
AIP_df = AIP_weekly('001980', start_date=start_date, end_date=end_date, fund_category='open', fixed_investment=1000, freq='Monday', df=True)
AIP_df

,净值日期,单位净值,日增长率,WeekDay,定投金额(本金),累计定投金额(本金),购买份额,累计份额,平均成本,累计收益,持有天数,年化收益率,累计收益率
0,2018-05-16,1.0000,0.00,Wednesday,0,0,0.000000,0.000000,NaN,NaN,0,NaN,NaN
1,2018-05-18,1.0002,0.00,Friday,0,0,0.000000,0.000000,NaN,NaN,0,NaN,NaN
2,2018-05-25,1.0003,0.00,Friday,0,0,0.000000,0.000000,NaN,NaN,0,NaN,NaN
3,2018-06-01,0.9999,0.00,Friday,0,0,0.000000,0.000000,NaN,NaN,0,NaN,NaN
4,2018-06-08,1.0008,0.00,Friday,0,0,0.000000,0.000000,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,2021-09-24,1.9603,-1.39,Friday,0,157000,0.000000,137947.604561,1.138113,113418.689220,1188,0.181818,0.722412
808,2021-09-27,1.9263,-1.73,Monday,1000,158000,519.129938,138466.734499,1.141068,108728.470665,1191,0.174070,0.688155
809,2021-09-28,1.9249,-0.07,Tuesday,0,158000,0.000000,138466.734499,1.141068,108534.617237,1192,0.173651,0.686928
810,2021-09-29,1.8725,-2.72,Wednesday,0,158000,0.000000,138466.734499,1.141068,101278.960349,1193,0.163626,0.641006


In [189]:
def AIP_Weekly_Plans(Freq, code, start_date, end_date, fund_category, fixed_investment, df=False):
    
    df = pd.DataFrame()
    
    for freq in Freq:
        df = df.append(AIP_weekly(code, start_date=start_date, end_date=end_date, fund_category=fund_category, fixed_investment=fixed_investment, freq=freq, df=False))
        
    return df

In [192]:
Freq = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

AIP_Weekly_Plans(Freq, '001980', start_date, end_date, fund_category='open', fixed_investment=1000, df=False)

,基金代码,持有天数,定投时间,定投金额,分投期数,总购买份额,平均成本,累计收益,累计收益率,年化收益率
Plan,001980,1194,Monday,1000,158.0,138466.734,1.141,106596.083,0.675,0.171
Plan,001980,1200,Tuesday,1000,162.0,141338.483,1.146,108083.707,0.667,0.168
Plan,001980,1234,Wednesday,1000,164.0,143250.605,1.145,109737.582,0.669,0.164
Plan,001980,1205,Thursday,1000,164.0,143136.130,1.146,109518.831,0.668,0.168
Plan,001980,1232,Friday,1000,162.0,141864.343,1.142,109088.573,0.673,0.165


In [196]:
AIP_df

,净值日期,单位净值,日增长率,WeekDay,定投金额(本金),累计定投金额(本金),购买份额,累计份额,平均成本,累计收益,持有天数,年化收益率,累计收益率
0,2018-05-16,1.0000,0.00,Wednesday,0,0,0.000000,0.000000,NaN,NaN,0,NaN,NaN
1,2018-05-18,1.0002,0.00,Friday,0,0,0.000000,0.000000,NaN,NaN,0,NaN,NaN
2,2018-05-25,1.0003,0.00,Friday,0,0,0.000000,0.000000,NaN,NaN,0,NaN,NaN
3,2018-06-01,0.9999,0.00,Friday,0,0,0.000000,0.000000,NaN,NaN,0,NaN,NaN
4,2018-06-08,1.0008,0.00,Friday,0,0,0.000000,0.000000,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,2021-09-24,1.9603,-1.39,Friday,0,157000,0.000000,137947.604561,1.138113,113418.689220,1188,0.181818,0.722412
808,2021-09-27,1.9263,-1.73,Monday,1000,158000,519.129938,138466.734499,1.141068,108728.470665,1191,0.174070,0.688155
809,2021-09-28,1.9249,-0.07,Tuesday,0,158000,0.000000,138466.734499,1.141068,108534.617237,1192,0.173651,0.686928
810,2021-09-29,1.8725,-2.72,Wednesday,0,158000,0.000000,138466.734499,1.141068,101278.960349,1193,0.163626,0.641006


In [257]:
def Max_AIP_weekly(code, start_date, end_date, fund_category, fixed_investment, Threshold=(-3.0, 2.0), df=False):
    
    fund_net_value = get_fund_net_worth(code, start_date=start_date, end_date=end_date, fund_category=fund_category)
    
    fund_net_value['WeekDay'] = pd.to_datetime(fund_net_value['净值日期']).dt.day_name()
    
    fund_net_value['定投金额(本金)'] = 0
    fund_net_value['累计定投金额(本金)'] = fund_net_value['定投金额(本金)'].cumsum()
        
    for i in range(len(fund_net_value['日增长率'])):
        if fund_net_value['日增长率'].values[i] <= Threshold[0]:
            fund_net_value['定投金额(本金)'][i] = fixed_investment
            fund_net_value['累计定投金额(本金)'] = fund_net_value['定投金额(本金)'].cumsum()
        elif (fund_net_value['日增长率'].values[i] >= Threshold[1]) & (fund_net_value['累计定投金额(本金)'].values[i-1] > fixed_investment):
            fund_net_value['定投金额(本金)'][i] = -fixed_investment
            fund_net_value['累计定投金额(本金)'] = fund_net_value['定投金额(本金)'].cumsum()

    fund_net_value['购买份额'] = fund_net_value['定投金额(本金)']/fund_net_value['单位净值']
    fund_net_value['累计份额'] = fund_net_value['购买份额'].cumsum()
    fund_net_value['平均成本'] = fund_net_value['累计定投金额(本金)']/fund_net_value['累计份额']

    fund_net_value['累计收益'] = (fund_net_value['单位净值'] - fund_net_value['平均成本']) * fund_net_value['累计份额']
    
    start_invest = fund_net_value['定投金额(本金)'].values.nonzero()[0][0]
    fund_net_value['持有天数'] = (fund_net_value['净值日期'] - fund_net_value['净值日期'][start_invest]).dt.days+1
    for i in range(len(fund_net_value['持有天数'])):
        if fund_net_value['持有天数'][i] < 0:
            fund_net_value['持有天数'][i] = 0
    fund_net_value['年化收益率'] = ((fund_net_value['累计收益'] + fund_net_value['累计定投金额(本金)'])/fund_net_value['累计定投金额(本金)'])**(365/fund_net_value['持有天数'])-1

    fund_net_value['累计收益率'] = fund_net_value['累计收益']/fund_net_value['累计定投金额(本金)']
    
    Stat_df = pd.DataFrame({
        '基金代码': code,
        '持有天数': fund_net_value['持有天数'].values[-1],
        '触发投资门槛(低买入)': Threshold[0],
        '触发投资门槛(高卖出)': Threshold[1],
        '单次金额': fixed_investment,
        '买入次数': len(fund_net_value[fund_net_value['定投金额(本金)'] == 1000]),
        '卖出次数': len(fund_net_value[fund_net_value['定投金额(本金)'] == -1000]),
        '总购买份额' : '%.3f' % fund_net_value['累计份额'].values[-1],
        '平均成本' : '%.3f' % fund_net_value['平均成本'].values[-1],
        '累计收益' : '%.3f' % fund_net_value['累计收益'].values[-1],
        '累计收益率' : '%.3f' % fund_net_value['累计收益率'].values[-1],
        '年化收益率' : '%.3f' % fund_net_value['年化收益率'].values[-1]
    }, index=['Plan'])
    
    if df:
        return fund_net_value
    else:
        return Stat_df

In [258]:
Max_AIP_weekly('001980', start_date=start_date, end_date=end_date, fund_category='open', fixed_investment=1000, df=False)

,基金代码,持有天数,触发投资门槛(低买入),触发投资门槛(高卖出),单次金额,买入次数,卖出次数,总购买份额,平均成本,累计收益,累计收益率,年化收益率
Plan,001980,1089,-3.0,2.0,1000,12,11,1368.136,0.731,1614.371,1.614,0.380


In [236]:
df = Max_AIP_weekly('001980', start_date=start_date, end_date=end_date, fund_category='open', fixed_investment=1000, df=True)

In [262]:
def Max_AIP_Weekly_Plans(code, start_date, end_date, fund_category, fixed_investment, upper_threshold, lower_threshold):
    
    df = pd.DataFrame()
    threshold_list = list(itertools.product(lower_threshold, upper_threshold))
    for i in range(len(threshold_list)):

        df = df.append(Max_AIP_weekly(code, start_date=start_date, end_date=end_date, fund_category=fund_category, fixed_investment=fixed_investment, 
                       Threshold=threshold_list[i], df=False))
        
    return df

In [266]:
upper_threshold = [1.0, 2.0, 3.0, 4.0, 5.0]
lower_threshold = [-1.0, -2.0, -3.0, -4.0, -5.0]

Max_AIP_Weekly_Plans('001980', start_date=start_date, end_date=end_date, fund_category='open', fixed_investment=1000, upper_threshold=upper_threshold, lower_threshold=lower_threshold)

,基金代码,持有天数,触发投资门槛(低买入),触发投资门槛(高卖出),单次金额,买入次数,卖出次数,总购买份额,平均成本,累计收益,累计收益率,年化收益率
Plan,001980,1185,-1.0,1.0,1000,130,126,5472.049,0.731,6456.538,1.614,0.344
Plan,001980,1185,-1.0,2.0,1000,130,51,73530.342,1.074,61509.130,0.779,0.194
Plan,001980,1185,-1.0,3.0,1000,130,20,100104.242,1.099,81289.196,0.739,0.186
Plan,001980,1185,-1.0,4.0,1000,130,4,115089.750,1.095,93925.003,0.745,0.187
Plan,001980,1185,-1.0,5.0,1000,130,2,117055.368,1.093,95681.102,0.748,0.188
Plan,001980,1163,-2.0,1.0,1000,43,42,1250.628,0.800,1389.825,1.390,0.314
Plan,001980,1163,-2.0,2.0,1000,43,42,3086.672,0.324,4898.321,4.898,0.745
Plan,001980,1163,-2.0,3.0,1000,43,19,22250.803,1.079,18519.060,0.772,0.197
Plan,001980,1163,-2.0,4.0,1000,43,4,36281.475,1.075,30330.270,0.778,0.198
Plan,001980,1163,-2.0,5.0,1000,43,2,38247.093,1.072,32086.370,0.783,0.199
